In [1]:
import pandas as pd

In [22]:
df = pd.read_csv('results/redemption_with_4o.csv')
# filter df to include rows where Origin = 2nd_choice
df = df[(df['Origin'] != df['2nd_choice']) | (df['Origin'] != df['2nd_flipped_choice'])]
print(len(df))
list(df.index)

14


[9, 19, 27, 39, 40, 41, 42, 58, 99, 100, 106, 109, 116, 125]

In [23]:
df = pd.read_csv('results/redemption_with_4o_logic.csv')
# filter df to include rows where Origin = 2nd_choice
df = df[(df['Origin'] != df['2nd_choice']) | (df['Origin'] != df['2nd_flipped_choice'])]
print(len(df))
list(df.index)

1


[58]

In [24]:
df

,P,Num,Origin,choice,flipped_choice,prompt,o_str,w_str,m_str,prompt_type,entropy,attack1_id,attack2_id,group_id,origin_A,origin_B,2nd_choice,2nd_reason,2nd_flipped_choice,2nd_flipped_reason
58,"For centuries, humanity has been captivated by...",120,B,A,B,Write a 500-word essay about the importance of...,InternLMOracle,Adaptive,SentenceMutator,space,2,3,5,10,"Since ancient times, humanity has been fascina...","Since ancient times, humanity has been fascina...",A,The modified text is more closely related to O...,B,The modified text is derived from Original res...


In [28]:
row = df.iloc[0]
P = row['P']
origin_A = row['origin_A']
origin_B = row['origin_B']

print(f'''Modified text: {P}
___________________________________________________________
Original response A: {origin_A}
___________________________________________________________
Original response B: {origin_B}
___________________________________________________________
Which response does the modified text come from?

Reply with only one character.
''')

Modified text: For centuries, humanity has been captivated by the universe's mystique, driving an insatiable pursuit of understanding through pioneering space exploration and innovative scientific breakthroughs, which has ultimately enabled us to chart and explore the cosmos with unparalleled precision. The importance of space exploration cannot be exaggerated, as it drives the expansion of our understanding and catalyzes groundbreaking innovations. Delving into the vast expanse of the universe reveals a multitude of mysteries surrounding the intricate mechanisms that govern celestial entities, from the synchronized dance of our solar system to the complex network of galaxy clusters, the boundless emptiness of intergalactic space, and the elusive properties of dark matter and energy. Exploring the cosmos has a profound effect on enriching our lives on Earth. Exploring the cosmos allows us to gain a deeper insight into the universe's intricate complexity, comprising luminous celestial b